In [1]:
import os
import sys

!git clone https://github.com/terryluan12/Genrify.git
!cd Genrify && make

sys.path.insert(0, "Genrify/src")

Cloning into 'Genrify'...
remote: Enumerating objects: 1143, done.
remote: Counting objects: 100% (531/531), done.
remote: Compressing objects: 100% (237/237), done.
remote: Total 1143 (delta 339), reused 434 (delta 286), pack-reused 612
Receiving objects: 100% (1143/1143), 7.79 MiB | 14.40 MiB/s, done.
Resolving deltas: 100% (728/728), done.
pip install -r requirements.txt
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=541002bae2f781c18d759037659e3b88b3c4da453da4294e73d519d865e15d7b
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [2]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
from datasources import download_datasets, convert_files
from preprocessing import preprocess
from cnn.testhandler.TestHandler import TestHandler

In [4]:
# Make sure to change subset_num to the subset which you are using
subset_num = 3 # not used for testing data
#Possible method values:
#"spec", "mel", "chroma", "mfcc" to retrieve exclusive training datasets
#"create_testing_data" to create testing data for the first time (not needed anymore due to option below)
#"test" to unzip already preprocessed testing data from Google Drive
wavTomp3Towav = True  #if testing out converting gtzan from wav to mp3 back to wav
method = "mfcc"

if method!="test":
    if method=="create_testing_data":
        !unzip /content/drive/MyDrive/"APS360 Team Project"/Test_Data/test_data_mp3.zip
        data_dir = "/content/test_data_mp3"
        output_dir = "/content/Genrify/src/datasources/test_data_wav"
        convert_files(data_dir, output_dir, wavToMp3=False)
    elif not os.path.isdir("Genrify/src/datasources/processed_data"):
        download_datasets("Genrify/src")
        if wavTomp3Towav:
            !mv /content/Genrify/src/datasources/Data/genres_original /content
            data_dir = "/content/genres_original"
            output1_dir = "/content/genres_original_mp3"
            convert_files(data_dir, output1_dir, wavToMp3=True)
            output2_dir = "/content/Genrify/src/datasources/Data/genres_original" #This will now contain the WAV files converted back from MP3
            convert_files(output1_dir, output2_dir, wavToMp3=False)

    preprocess(subset_num, method, "Genrify/src")

else:
    #Unzipping already preprocessed testing data
    !unzip -q '/content/drive/MyDrive/APS360 Team Project/Test_Data/test_mel.zip' -d '/'
    !unzip -q '/content/drive/MyDrive/APS360 Team Project/Test_Data/test_spectrogram.zip' -d '/'
    !unzip -q '/content/drive/MyDrive/APS360 Team Project/Test_Data/test_chroma.zip' -d '/'
    !unzip -q '/content/drive/MyDrive/APS360 Team Project/Test_Data/test_mfcc.zip' -d '/'
    test_handler = TestHandler(batch_size=1, num_workers=1)

Downloaded Datasources
Conversion successful: /content/genres_original/metal/metal.00095.wav -> /content/genres_original_mp3/metal/metal.00095.mp3
Conversion successful: /content/genres_original/metal/metal.00065.wav -> /content/genres_original_mp3/metal/metal.00065.mp3
Conversion successful: /content/genres_original/metal/metal.00036.wav -> /content/genres_original_mp3/metal/metal.00036.mp3
Conversion successful: /content/genres_original/metal/metal.00087.wav -> /content/genres_original_mp3/metal/metal.00087.mp3
Conversion successful: /content/genres_original/metal/metal.00032.wav -> /content/genres_original_mp3/metal/metal.00032.mp3
Conversion successful: /content/genres_original/metal/metal.00052.wav -> /content/genres_original_mp3/metal/metal.00052.mp3
Conversion successful: /content/genres_original/metal/metal.00080.wav -> /content/genres_original_mp3/metal/metal.00080.mp3
Conversion successful: /content/genres_original/metal/metal.00040.wav -> /content/genres_original_mp3/metal/m

In [5]:
from cnn import mfcc_model, mfcc_training
from utils import plot
from cnn.datahandler.DataHandler import DataHandler
import os
import torch

In [ ]:
root_dir = "/content/Genrify/src"
mfcc_data_dir = os.path.join(root_dir, "datasources", "mfcc")
mfcc_data_handler = DataHandler(mfcc_data_dir, batch_size=32, num_workers=4)
for ss, g in [(5, 0.3)]:
    for lr in [5e-5, 1e-4, 2e-4, 3e-4, 5e-4, 1e-3, 2e-3]:
        num_epochs = 60
        mfcc_cnn = mfcc_model.MFCC_Resnet(num_classes=10)
        print(f"Learning rate: {lr}, Subset size: {ss}, Gamma: {g}")
        best_epoch = mfcc_training.train(mfcc_cnn, mfcc_data_handler.train_loader, mfcc_data_handler.val_loader, num_epochs, lr, 32, ss, g, patience=5)
        model_path = mfcc_training.get_model_name(mfcc_cnn.name, 32, lr, best_epoch)
        %matplotlib inline
        # plot.plot_training_curve(model_path)
        # plot.plot_confusion_matrix(model_path, range(10))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can als

Learning rate: 5e-05, Subset size: 5, Gamma: 0.3
Epoch [1/60], Train Loss: 1.2498, Train Accuracy: 56.32%, Val Loss: 0.9747, Val Accuracy: 65.14%
Epoch [2/60], Train Loss: 0.6926, Train Accuracy: 77.11%, Val Loss: 0.6665, Val Accuracy: 77.19%
Epoch [3/60], Train Loss: 0.3662, Train Accuracy: 89.15%, Val Loss: 0.4367, Val Accuracy: 85.19%
Epoch [4/60], Train Loss: 0.1614, Train Accuracy: 96.03%, Val Loss: 0.3070, Val Accuracy: 90.25%
Epoch [5/60], Train Loss: 0.0861, Train Accuracy: 98.48%, Val Loss: 0.3298, Val Accuracy: 89.10%
Epoch [6/60], Train Loss: 0.0299, Train Accuracy: 99.74%, Val Loss: 0.1872, Val Accuracy: 93.71%
Epoch [7/60], Train Loss: 0.0179, Train Accuracy: 99.95%, Val Loss: 0.1737, Val Accuracy: 94.35%
Epoch [8/60], Train Loss: 0.0145, Train Accuracy: 99.94%, Val Loss: 0.1628, Val Accuracy: 94.67%
Epoch [9/60], Train Loss: 0.0121, Train Accuracy: 99.98%, Val Loss: 0.1635, Val Accuracy: 94.43%
Epoch [10/60], Train Loss: 0.0149, Train Accuracy: 99.96%, Val Loss: 0.1684, V

In [ ]:
!zip -r '/content/drive/MyDrive/mp3_train_mfcc.zip' '/content/Genrify/src/datasources/mfcc'

In [ ]:
!unzip -q '/content/drive/MyDrive/mp3_train_mfcc.zip' -d '/'